In [3]:
#http://paper.people.com.cn/rmrb/page/2018-03/25/01/rmrb2018032501.pdf
import urllib
import datetime
import requests
import os
import sys
import glob
import re 

from PyPDF2 import PdfFileReader,PdfFileWriter
from bs4 import BeautifulSoup  
from datetime import timedelta, date



def getFileName(filepath):
    file_list = []
    for root,dirs,files in os.walk(filepath):
        for filespath in files:
            # print(os.path.join(root,filespath))
            file_list.append(os.path.join(root,filespath))
    return file_list

def date_range(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def download_separate_pdf(year, month, day):
    url = "http://paper.people.com.cn/rmrb/html/2018-03/25/nbs.D110000renmrb_01.htm"
    pdf_url01 = "http://paper.people.com.cn/rmrb/page/2018-03/25/01/rmrb2018032501.pdf"
    pdf_url02 = "http://paper.people.com.cn/rmrb/page/2018-03/25/02/rmrb2018032502.pdf"

    # Generate the first page html
    paper_people_base = "http://paper.people.com.cn/rmrb/html/"
    paper_people_download_base = "http://paper.people.com.cn/rmrb/"

    current_time = year + '-' + month + '/' + day + "/nbs.D110000renmrb_01.htm"
    url = paper_people_base + current_time
    print url
    # Generate the pdf list for donwloading
    #web_raw = urllib.urlopen(url)
    html = requests.get(url) 
    html.encoding = 'utf-8' #这一行是将编码转为utf-8否则中文会显示乱码。  

    #print type(html)
    #print html.text
    #print type(web_raw)
    #print web_raw.read()
    soup = BeautifulSoup(html.text, "html.parser")
    #print soup.prettify()



    #right_title_pdf_class = soup.find_all('div', class_="right_title-pdf",)
    #print right_title_pdf_class
    #print type(right_title_pdf_class)
    a_href_list = soup.select('a[href^="../../../page"]') # '^' 匹配字符串开头
    #print a_href_list 
    #print type(a_href_list)
    #print len(a_href_list)
    #for each_a_href in a_href_list:
    #    print each_a_href.get('href')[-38:]

    download_pdf_list = [each_a_href.get('href')[-37:] for each_a_href in a_href_list]
    #print download_pdf_list

    download_pdf_list_remove_duplicate = []
    for i in download_pdf_list:
      if i not in download_pdf_list_remove_duplicate:
        download_pdf_list_remove_duplicate.append(i)

    print download_pdf_list_remove_duplicate

    for current_pdf in download_pdf_list_remove_duplicate:
        current_file_name = current_pdf.split('/')[-1]
        current_url = paper_people_download_base + current_pdf
        print "Downloading " + current_file_name + " now."
        response = urllib.urlopen(current_url)
        file = open(current_file_name, 'wb')
        file.write(response.read())
        file.close()
        print "Finished."
        print "---------------"


def merge_separate_pdf_to_one(year, month, day):
    filepath = os.getcwd()
    #infnList = ['rmrb2018032501.pdf','rmrb2018032502.pdf']
    #print infnList
    current_file_name_list = getFileName(filepath)
    current_file_name_filter = "rmrb" + year + month + day
    infnList = []
    for each_file in current_file_name_list:
        each_file_name = each_file.split('/')[-1]
        if each_file.split('/')[-1].startswith(current_file_name_filter):
            infnList.append(each_file_name)

    infnList =  sorted(infnList)
    print infnList

    pdf_output = PdfFileWriter()
    for infn in infnList:
        print infn
        pdf_input = PdfFileReader(open(infn, 'rb'), strict=False)
        # 获取 pdf 共用多少页
        page_count = pdf_input.getNumPages()
        print(page_count)
        for i in range(page_count):
            pdf_output.addPage(pdf_input.getPage(i))

    summaryName = str(year) + "." + str(month) +"." + str(day) + '_People_Summary.pdf'
    pdf_output.write(open(summaryName, 'wb'))

#select_day = "2018.04.30"
#[year, month, day] = select_day.split(".")
# now = datetime.datetime.now()
# year = str(now.year)
# month = str(now.month).zfill(2)
# day = str(now.day).zfill(2)

start_date = date(2018, 4, 30)
end_date = date(2018, 5, 1)
end_date = end_date  + datetime.timedelta(days=1)
number_of_days = abs(end_date - start_date).days
iterator_of_days = 0

for single_date in date_range(start_date, end_date):
    iterator_of_days = iterator_of_days + 1
    print single_date.strftime("%Y-%m-%d")
    t = single_date.strftime("%Y-%m-%d")
    [year, month, day] = t.split("-")
    print year, month, day
    download_separate_pdf(year, month, day)  
    merge_separate_pdf_to_one(year, month, day)
    print"Completed Download " + year + "-"+ month + "-" + day
    print"=====================" + str(iterator_of_days) +"/" + str(number_of_days) + "==============================="

#print("Completed")

print ("Competed Download All!")


2018-04-30
2018 04 30
http://paper.people.com.cn/rmrb/html/2018-04/30/nbs.D110000renmrb_01.htm
[u'page/2018-04/30/01/rmrb2018043001.pdf', u'page/2018-04/30/02/rmrb2018043002.pdf', u'page/2018-04/30/03/rmrb2018043003.pdf', u'page/2018-04/30/04/rmrb2018043004.pdf', u'page/2018-04/30/05/rmrb2018043005.pdf', u'page/2018-04/30/06/rmrb2018043006.pdf', u'page/2018-04/30/07/rmrb2018043007.pdf', u'page/2018-04/30/08/rmrb2018043008.pdf']
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
['rmrb2018043001.pdf', 'rmrb2018043002.pdf', 'rmrb2018043003.pdf', 'rmrb2018043004.pdf', 'rmrb2018043005.pdf', 'rmrb2018043006.pdf', 'rmrb2018043007.pdf', 'rmrb2018043008.pdf']
rmrb2018043001.pdf
1
rmrb2018043002.pdf
1
rmrb2018043003.pdf
1
rmrb2018043004.pdf
1
rmrb2018043005.pdf
1
rmrb2018043006.pdf
1
rmrb2018043007.pdf
1
rmrb2018043008.pdf
1


Completed Download 2018-04-30
=====================1/2===============================
2018-05-01
2018 05 01
http://paper.people.com.cn/rmrb/html/2018-05/01/nbs.D110000renmrb_01.htm
[u'page/2018-05/01/01/rmrb2018050101.pdf', u'page/2018-05/01/02/rmrb2018050102.pdf', u'page/2018-05/01/03/rmrb2018050103.pdf', u'page/2018-05/01/04/rmrb2018050104.pdf', u'page/2018-05/01/05/rmrb2018050105.pdf', u'page/2018-05/01/06/rmrb2018050106.pdf', u'page/2018-05/01/07/rmrb2018050107.pdf', u'page/2018-05/01/08/rmrb2018050108.pdf']
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
Finished.
---------------
['rmrb2018050101.pdf', 'rmrb2018050102.pdf', 'rmrb2018050103.pdf', 'rmrb2018050104.pdf', 'rmrb2018050105.pdf', 'rmrb2018050106.pdf', 'rmrb2018050107.pdf', 'rmrb2018050108.pdf']
rmrb2018050101.pdf
1
rmrb2018050102.pdf
1
rmrb2018050103.pdf
1
rmrb2018050104.pdf
1
rmrb201805010

Completed Download 2018-05-01
=====================2/2===============================
Competed Download All!
